<a href="https://colab.research.google.com/github/lordleoo/matpower/blob/master/MyTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%
import sys, os
import datetime as dt
sys.version

if os.environ["COLAB_GPU"]!='1':
    sys.exit('On the right bottom corner, click on Python 3 >> "Change runtime type" >> "T4 GPU". This is much faster and stable')
else:
    print('GPU is in used. Good on you ;) learning your listen.')

!apt-get update && apt-get install -y espeak-ng
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#%%
import datetime as dt
!pip install TTS
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# FOR ARABIC TTS, which doesn't work so far
#!pip freeze
#### https://docs.coqui.ai/en/latest/models/xtts.html
#You can check all supported languages with the following command: # RUN THIS (BELOW) IN THE TERMINAL, NOT IN THE JUPYTER NOTEBOOK
# tts --model_name tts_models/multilingual/multi-dataset/xtts_v2 --list_language_idx

#You can check all Coqui available speakers with the following command:
# tts --model_name tts_models/multilingual/multi-dataset/xtts_v2 --list_speaker_idx


In [ ]:
#%% If you restart a session, start from here. no need to reinstall TTS and ...
import sys, os, torch
import datetime as dt
from TTS.api import TTS     # https://docs.coqui.ai/en/latest/models/xtts.html
print('Python version is:',sys.version)
print('Currently working from:', os.getcwd())
os.listdir()
time2hms = lambda xx: '%i:%0.2i:%0.2i.%0.3i' %(xx//3600, (xx%3600)//60, (xx%60)//1, (xx%1)*1000)
tictoc   = lambda tstart: 'Execution time: %s' %time2hms((xx:=(dt.datetime.now() - tstart)).total_seconds()+(xx.microseconds*1e-6))

print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#%%
import torch
from TTS.api import TTS
import datetime as dt

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Init TTS with a specific model
# You can find available models by running: print(TTS().list_models())
tts = TTS(model_name="tts_models/en/ljspeech/vits", progress_bar=True).to(device)

print("TTS model initialized successfully!")
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#%%
tts = TTS(model_name="tts_models/en/ljspeech/vits", progress_bar=True).to(device)
print(tts.synthesizer.tts_config)
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#%%
import datetime as dt
try:
    from TTS.tts.utils.text.phonemizers import espeak_wrapper
    print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
except Exception as e:
    print(f"Error importing espeak_wrapper: {e}")

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print("Available models:", TTS().list_models(),'\n'*2)
print(TTS().list_models())
# Init TTS
# Use a model from the list above, for example 'tts_models/en/ljspeech/vits'
Lang = 2; # Arabic doesn't work
try:
    if  Lang == 1: # ARABIC
        model_name = "tts_models/ar/multi-dataset/vits" # Example Arabic model name
        tts_arabic = TTS(model_name=model_name, progress_bar=True).to(device)
        print(f'Today\'s special dish: Arabic TTS model ({model_name})')
        tts_arabic.tts_to_file(text="هذا نص تجريبي باللغة العربية", file_path="arabic_output.wav")
    elif Lang == 2: # ENGLISH
        model_name="tts_models/en/ljspeech/vits"
        tts = TTS(model_name=model_name, progress_bar=True).to(device)
        print('Today\'s special dish: English TTS model')
    elif Lang==3:
        model_name="tts_models/fr/css10/vits"
        tts = TTS(model_name=model_name, progress_bar=True).to(device)
        print('Today\'s special dish: French TTS model')
except Exception as e: # FRENCH
    print(f"Error initializing TTS model:\n{e}")
    print(f"Please check if the model name '{model_name}' is available in the list above.")
    raise e

print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
#%% Fetch the TXT file and split to pages
import os, re
import datetime as dt
# filename = r"E:\Documents\LIST Personal\Hobbies\Reading Books\The selfish gene - Audiobook\Richard Dawkins - The Selfish Gene.txt"
filename = r"Final Exam Subjects Practice.txt"

with open(filename, 'r') as f:
  full_text = f.read()

#full_text=re.sub('(\r\n){2,}','\r\n\r\n',full_text0)
text_list = [yy for xx in full_text.split('\n\n') if len(yy:=xx.strip())>0]
text_len = len(full_text.replace('\n\n',' '))

print("Generating speech...")
start_from = 0;
os.makedirs('Audios', exist_ok=True)
str_file_len = len(str(len(text_list)))
str_line_len = len(str(text_len))
char_fininshed=0;
line_finished=0
openings = []

print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#%% CORE TTS

for itr,txt in enumerate(text_list):
    openings.append([itr,line_finished+itr+1,txt.split('\n')[0]])
    split_len = len(txt)
    char_fininshed+=split_len
    line_finished+=len(txt.split('\n'))
    prcnt_finished = (char_fininshed/text_len * 100)
    # start_from = max(itr-1,0)

    if itr<start_from:
        continue
    # Run TTS
    #  Since this model is multi-speaker please input speaker_idx according to the speaker you want to use.
    tim = dt.datetime.now()
    print('\nRecording file#%4i/%i of length %4i characters. %4.1f%% done @ %s.' %(itr,
                                                                                    len(text_list),
                                                                                    len(txt),
                                                                                    prcnt_finished,
                                                                                    tim.strftime('%A %d/%B/%Y @ %H:%M:%S')))

    #print('Finished file #%i within %s' %(itr,tictoc(tim)))
    output_file_name = os.path.join('Audios', f'{itr:>0{str_file_len}}.wav')
    #####################################################################
    try:
        tts.tts_to_file(text=txt, file_path=output_file_name, speaker_idx=0)
    #If you face problems here, just go few cells up and re-import TTS.
    #Don't forget to update start_from
    except Exception as e:
        print(f"Error in text#{itr} = {txt}")
        print(f"Error in TTS: {e}")
        raise
    #####################################################################
    print('Output saved as: %s' %output_file_name)
    #sys.exit('Breakpoint')

# You can then download the output.wav file to your local machine.
# from google.colab import files
# files.download("output.wav")

# openings_str = ['%i) %s' %tuple(xx) for xx in openings]
openings_str = [f'{xx[0]:>0{str_file_len}})[Line#{xx[1]:>0{str_line_len}}] {xx[2]}' for xx in openings]
#FU.WriteTXT(filename.replace('.txt','-INDEX.txt'),'\n'.join(openings_str),overwrite=True)
output_filename = filename.replace('.txt', '-INDEX.txt')
# Write the contents of openings_str to the file
with open(output_filename, 'w') as f:
    for line in openings_str:
        f.write(line + '\n')

print(f"Contents of openings_str written to {output_filename}")
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
#%% Download WAV Files
from google.colab import files
import os

# Get a list of all files in the "Audios" directory that end with ".wav"
audio_files = [f for f in os.listdir('Audios') if f.endswith('.wav')]

# Download each file
for file_name in audio_files:
  print(f"Downloading {file_name}...")
  files.download(os.path.join('Audios', file_name))

print("\n\n\nFinished downloading files @%s." %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

**Caution:** The following command will permanently delete all `.wav` files in the `Audios` folder. Make sure you have a backup or are certain you want to delete these files before running it.

In [ ]:
#%% Move WAV files from main folder to /Audios folder. unnecessary now.
import os

# Change to the Audios directory
%cd /content/Audios

# Use a shell command to delete all .wav files
!mv *.wav Audios/

print(f"\n\n\nFinished moving files @{dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

In [ ]:
#%% Remove WAV files or MP3 files
import os

# Change to the Audios directory
%cd /content/Audios

# Use a shell command to delete all .wav files
resp = input('Are you sure? (y/n)\t')
if resp[0].lower() == 'y':
    if True:
        !rm ./Audios/*.wav
        #!rm
        pass
    else:
        !rm ./Audios/MP3/*.mp3
        pass

print(f"\n\n\nFinished deleting files @{dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

# Task
Write a python code in Google Colab to find the length of all ".wav" files in the folder "Audios/", create a table with filenames and their lengths, and then convert each ".wav" file to ".mp3".

## Install necessary libraries

### Subtask:
Install libraries required for audio processing (e.g., `pydub`).


**Reasoning**:
Install the `pydub` and `ffmpeg` libraries to handle audio processing and conversion.



In [ ]:
#%% Prepare to conver to MP3 by installing PyDub
!pip install pydub
!apt-get update && apt-get install -y ffmpeg

print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

## List audio files

### Subtask:
Get a list of all `.wav` files in the "Audios/" directory.


**Reasoning**:
The subtask is to get a list of all `.wav` files in the "Audios/" directory. This can be achieved by listing the files in the directory and filtering for those ending with ".wav".



**Reasoning**:
The previous attempt failed because the directory "Audios/" was not found. I need to verify if the directory exists and create it if it doesn't before trying to list the files within it.



In [ ]:
#%% Sort WAV Files and create a list
import os
import numpy as np

audio_dir = "Audios"
os.makedirs(audio_dir, exist_ok=True)

wav_files = np.sort([f for f in os.listdir(audio_dir) if f.endswith('.wav')])
print('%i files were found:\n%s' %(len(wav_files), wav_files))
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

## Get audio lengths

### Subtask:
Iterate through the list of `.wav` files, get the length of each audio file, and store the filenames and lengths.


**Reasoning**:
Iterate through the list of wav files, get the length of each audio file, and store the filenames and lengths as instructed.



In [ ]:
#%% Survey clips lengths
from pydub import AudioSegment
time2hms = lambda xx: '%i:%0.2i:%0.2i,%0.3i' %(xx//3600, (xx%3600)//60, (xx%60)//1, (xx%1)*1000)

audio_lengths = []
cumsum = 0
cumsum_hms = '00:00:00,000'
for filename,opn in zip(wav_files,openings):
    file_path           = os.path.join(audio_dir, filename)
    audio               = AudioSegment.from_wav(file_path)
    length_in_seconds   = audio.duration_seconds
    length_in_seconds_hms = time2hms(length_in_seconds)
    audio_lengths.append((filename,
                          length_in_seconds,
                          length_in_seconds_hms,
                          cumsum,
                          cumsum_hms,
                          *opn))
    cumsum += length_in_seconds
    cumsum_hms = time2hms(cumsum)

print(audio_lengths)
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

## Create a table

### Subtask:
Create a pandas DataFrame to store the filenames and their corresponding lengths and display the table.


**Reasoning**:
Create a pandas DataFrame from the list of audio lengths and display it.



In [ ]:
#%% Create a table of timestamps / lengths of clips, to create an index
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_audio_lengths = pd.DataFrame(audio_lengths, columns=['Filename',
                                                        'Length',
                                                        'Length(h:m:s)',
                                                        'CumSum',
                                                        'CumSum(h:m:s)',
                                                        'itr',
                                                        'line#',
                                                        'line_txt'])
display(df_audio_lengths)
print('')
print(df_audio_lengths)

# Export to a text file (CSV format)
df_audio_lengths.to_csv('audio_lengths.txt', sep='\t', index=False)

# Export to an Excel file
df_audio_lengths.to_excel('audio_lengths.xlsx', index=False)

print("DataFrame exported to audio_lengths.txt and audio_lengths.xlsx")
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

## Convert to mp3

### Subtask:
Iterate through the list of `.wav` files and convert each file to `.mp3` format.


**Reasoning**:
Import the necessary library and iterate through the audio files to convert them to MP3 format.



In [ ]:
#%%  Prepare to convert WAV to MP3
from pydub import AudioSegment

wav_dir = "Audios/" # Define the directory name
mp3_dir = "Audios/MP3" # Define the directory name
os.makedirs(mp3_dir, exist_ok=True)

for filename, length, _, _,_,_,_,_ in audio_lengths:
    wav_path = os.path.join(wav_dir, filename)
    mp3_filename = filename.replace('.wav', '.mp3')
    mp3_path = os.path.join(mp3_dir, mp3_filename)

    # Load the WAV file
    audio = AudioSegment.from_wav(wav_path)

    # Export to MP3
    audio.export(mp3_path, format='mp3')

    print(f"Converted {filename} to {mp3_filename} out of {len(audio_lengths)}")

print("All files converted!")
print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#%% Download MP3 files
from google.colab import files
import os
import datetime as dt # Import datetime for the print statement

# Get a list of all files in the "Audios/MP3" directory that end with ".mp3"
audio_files = [f for f in os.listdir('Audios/MP3') if f.endswith('.mp3')]

# Download each file
for file_name in audio_files:
  print(f"Downloading {file_name}...")
  files.download(os.path.join('Audios/MP3', file_name)) # Corrected download path

print("\n\n\nFinished downloading files @%s." %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
import FileUtilities as FU
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
xls_file = r"E:\Documents\LIST Personal\Hobbies\Reading Books\Axelrod-Evolution of cooperation\audio_lengths.xlsx"
lengths0 = FU.ReadXLS(xls_file,index_col=0)
lengths  = lengths0['Content']['Sheet1']
lengths_hms = [FU.time2hms(xx) for xx in lengths.values[:,0]]
milestones_hms = [FU.time2hms(xx) for xx in lengths.values[:,1]]
lengths.insert(lengths.shape[1],'length_hms',lengths_hms)
lengths.insert(lengths.shape[-1],'Milestone',milestones_hms)

with pd.ExcelWriter(xls_file, engine="openpyxl", mode='a', if_sheet_exists='new') as writer:
     lengths.to_excel(writer, sheet_name='Sheet2')


## Summary:

### Data Analysis Key Findings

*   The necessary libraries (`pydub` and `ffmpeg`) were successfully installed or confirmed to be already present.
*   The "Audios/" directory was created as it did not exist initially.
*   No `.wav` files were found in the "Audios/" directory, resulting in empty lists for `wav_files` and `audio_lengths`.
*   A pandas DataFrame was successfully created but was empty due to the lack of audio file data.
*   The code for converting `.wav` to `.mp3` was executed successfully, but since the list of audio files was empty, no conversions actually occurred.

### Insights or Next Steps

*   Populate the "Audios/" directory with `.wav` files to proceed with the analysis and conversion.
*   Rerun the steps to list files, get lengths, create the table, and convert to MP3 after adding audio files.


In [ ]:
# Assuming 'device' and 'TTS' are already imported and initialized from previous cells

# List available 🐸TTS models to find an Arabic one
print("Available models:", TTS().list_models(),'\n'*2)

# Choose an Arabic model from the list, for example 'tts_models/ar/multi-dataset/vits'
# Replace with the actual model name you find in the list
arabic_model_name = "tts_models/ar/multi-dataset/vits" # Example Arabic model name

try:
    # Init TTS with the Arabic model
    tts_arabic = TTS(model_name=arabic_model_name, progress_bar=True).to(device)
    print(f'Today\'s special dish: Arabic TTS model ({arabic_model_name})')

    # Now you can use tts_arabic to synthesize speech in Arabic
    # For example:
    # tts_arabic.tts_to_file(text="هذا نص تجريبي باللغة العربية", file_path="arabic_output.wav")

except Exception as e:
    print(f"Error initializing Arabic TTS model: {e}")
    print(f"Please check if the model name '{arabic_model_name}' is available in the list above.")

print("\n\n\n@%s: You can proceed to the next cell" %dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))